In [ ]:
import sys
sys.path.append('../util')

from util import read_spreadsheet

In [ ]:
#NW DEPARTMENT
df = read_spreadsheet(sheet_id="1-WuaaWKIHJfGQhDXVSwG-v2qGqxQYVqqk7lFeHyxEzk")

In [ ]:
df.columns

In [ ]:
duration = df['standard duration ']

In [ ]:
df['standard duration '].to_csv('duration.csv', index=False)

In [ ]:
import pandas as pd
# Function to convert HH:MM:SS.mmm to total seconds
def convert_to_seconds(time_str):
    if pd.isna(time_str) or not str(time_str).strip(): 
        print(time_str)  # Check for NaN, None, or empty string
        return 0  # Treat empty or NaN time as 0 seconds
    
    # Split the time string into hours, minutes, and seconds.milliseconds
    h, m, s = str(time_str).split(':')
    
    # Further split the seconds part to separate seconds and milliseconds
    s, ms = map(float, s.split('.')) if '.' in s else (int(s), 0.0)
    
    # Convert hours, minutes, and seconds to total seconds
    return int(h) * 3600 + int(m) * 60 + s + ms / 1000

In [ ]:
# Convert all time durations to seconds and sum them
total_seconds = sum(convert_to_seconds(td) for td in duration)

hours = int(total_seconds // 3600)
minutes = int((total_seconds % 3600) // 60)
seconds = total_seconds % 60

# Format the result as HH:MM:SS
total_duration_str = f"{hours:02}:{minutes:02}:{seconds:02}"

print(f"Total Duration: {total_duration_str}")

In [ ]:
df1 = df

In [ ]:
df1.columns

In [ ]:
df2 = pd.read_csv("dept_files/STT_CS.csv")

In [ ]:
df2.columns

In [ ]:
df2.head()

In [ ]:
merged_df = pd.merge(df1, df2, on='Audio File (Original)', how='left')

In [ ]:
merged_df.head()

In [ ]:
merged_df.columns

In [ ]:
# Correct way to select specific columns
split_df = merged_df[['Audio File (Original)', 'standard duration ', 'split duration', 'Added in Prodegy ']]

In [ ]:
split_df.head()

In [ ]:
split_df.to_csv("merged_data/STT_CS.csv", index=False)

In [ ]:
from datetime import datetime, timedelta    
def subtract_durations(row):
    if pd.isna(row['split duration']):
        return ''
    else:
        # Convert to datetime objects1SjlnWsr6YVhBIW6EaS0ITmboDmEuj2VnxGRNL50OPx4
        fmt = '%H:%M:%S.%f' if '.' in row['split duration'] else '%H:%M:%S'
        
        # Convert to datetime.time objects
        standard_time = datetime.strptime(row['standard duration '], '%H:%M:%S').time()
        split_time = datetime.strptime(row['split duration'], fmt).time()
        
        # Combine time with a base date to perform subtraction
        base_date = datetime(2000, 1, 1)  # Arbitrary base date
        standard_datetime = datetime.combine(base_date, standard_time)
        split_datetime = datetime.combine(base_date, split_time)
        
        # Calculate remaining duration
        remaining_time = standard_datetime - split_datetime
        if remaining_time < timedelta(0):
            print(f"Negative time difference: {remaining_time}", row['Audio File (Original)'])
            # If negative, adjust by adding one day
            remaining_time = timedelta(0)
        
        # Convert remaining time to hh:mm:ss format
        remaining_hours, remainder = divmod(remaining_time.seconds, 3600)
        remaining_minutes, remaining_seconds = divmod(remainder, 60)
        
        return f"{remaining_hours:02}:{remaining_minutes:02}:{remaining_seconds:02}"


In [ ]:
split_df['remaining_duration'] = split_df.apply(subtract_durations, axis=1)


In [ ]:
final_df = split_df[['Audio File (Original)', 'standard duration ', 'split duration',  'Added in Prodegy ']]


In [ ]:
final_df.to_csv("merged_data/STT_CS.csv", index=False)

In [ ]:
split_duration = final_df['split duration']

In [ ]:
#Convert all time durations to seconds and sum them
total_seconds = sum(convert_to_seconds(td) for td in split_duration if td)

# Convert the total seconds back to HH:MM:SS format
hours = int(total_seconds // 3600)
minutes = int((total_seconds % 3600) // 60)
seconds = total_seconds % 60
# Format the result as HH:MM:SS
total_duration_str = f"{hours:02}:{minutes:02}:{seconds:02}"

print(f"Total split Duration: {total_duration_str}")

In [ ]:
loss_duration = final_df['remaining_duration']

In [ ]:
#Convert all time durations to seconds and sum them
total_seconds = sum(convert_to_seconds(td) for td in loss_duration if td)

# Convert the total seconds back to HH:MM:SS format
hours = int(total_seconds // 3600)
minutes = int((total_seconds % 3600) // 60)
seconds = total_seconds % 60
# Format the result as HH:MM:SS
total_duration_str = f"{hours:02}:{minutes:02}:{seconds:02}"

print(f"Total loss Duration: {total_duration_str}")

In [ ]:
total_uploaded_duration_seconds = sum(
    convert_to_seconds(row['standard duration ']) 
    for _, row in final_df.iterrows() 
    if pd.notna(row['split duration']) and str(row['split duration']).strip()
)

# Convert total seconds back to HH:MM:SS format
total_hours, remainder = divmod(total_uploaded_duration_seconds, 3600)
total_minutes, total_seconds = divmod(remainder, 60)

total_uploaded_duration = f"{int(total_hours):02}:{int(total_minutes):02}:{int(total_seconds):02}"

print("Total Uploaded Duration:", total_uploaded_duration)

In [ ]:
# Calculate total uploaded duration for rows where 'Pecha Tools' is 'uploaded'
total_uploaded_duration_seconds = sum(
    convert_to_seconds(row['standard duration ']) 
    for _, row in final_df.iterrows() 
    if  row['Added in Prodegy '] == True
)

# Convert total seconds back to HH:MM:SS format
total_hours, remainder = divmod(total_uploaded_duration_seconds, 3600)
total_minutes, total_seconds = divmod(remainder, 60)

total_uploaded_duration = f"{int(total_hours):02}:{int(total_minutes):02}:{int(total_seconds):02}"

print("Total Uploaded Duration:", total_uploaded_duration)

In [ ]:
# Calculate total uploaded duration for rows where 'Pecha Tools' is 'uploaded'
total_uploaded_duration_seconds = sum(
    convert_to_seconds(row['split duration']) 
    for _, row in final_df.iterrows() 
    if  row['Added in Prodegy '] == True
)

# Convert total seconds back to HH:MM:SS format
total_hours, remainder = divmod(total_uploaded_duration_seconds, 3600)
total_minutes, total_seconds = divmod(remainder, 60)

total_uploaded_duration = f"{int(total_hours):02}:{int(total_minutes):02}:{int(total_seconds):02}"

print("Total split Duration:", total_uploaded_duration)

In [ ]:
all_split_df = pd.read_csv("dept_files/STT_CS.csv")

In [ ]:
total_uploaded_duration_seconds = sum(
    convert_to_seconds(row['split duration']) 
    for _, row in all_split_df.iterrows() 
)

# Convert total seconds back to HH:MM:SS format
total_hours, remainder = divmod(total_uploaded_duration_seconds, 3600)
total_minutes, total_seconds = divmod(remainder, 60)

total_uploaded_duration = f"{int(total_hours):02}:{int(total_minutes):02}:{int(total_seconds):02}"

print("Total after split Duration:", total_uploaded_duration)